In [10]:
import pickle

mask_model = pickle.load(open('check_mask.sav', 'rb'))
wmask_model = pickle.load(open('w_mask.sav', 'rb'))
womask_model = pickle.load(open('wo_mask.sav', 'rb'))

In [11]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

import mtcnn
from PIL import Image

# Suppress warnings
import logging
logging.getLogger('tensorflow').disabled = True

# Force GPU mode
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [12]:
from keras_facenet import FaceNet
embedder = FaceNet()

def create_embedding(face):
    
    # Make prediction to get embedding
    detections = embedder.extract(face)
    
    # Return embedding (flatten it first)
    return detections[0]['embedding']

In [7]:
in_encoder1 = Normalizer(norm = 'l2')

data1 = np.load('lt3.npz')
train_Y1 = data1['arr_1']
out_encoder1 = LabelEncoder()
out_encoder1.fit(train_Y1)

LabelEncoder()

In [8]:
in_encoder2 = Normalizer(norm = 'l2')

data2 = np.load('lt3_mask.npz')
train_Y2 = data2['arr_1']
out_encoder2 = LabelEncoder()
out_encoder2.fit(train_Y2)

LabelEncoder()

In [9]:
in_encoder3 = Normalizer(norm = 'l2')

data3 = np.load('lt3_maskornot.npz')
train_Y3 = data3['arr_1']
out_encoder3 = LabelEncoder()
out_encoder3.fit(train_Y3)

LabelEncoder()

In [ ]:
import cv2

classifier = cv2.CascadeClassifier('models/haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

while True:

    ret, frame = video_capture.read()

    faces = classifier.detectMultiScale(frame)

    for result in faces:
        
        x, y, w, h = result
        x1, y1 = x + w, y + h
        
        r = max(w, h) / 2
        centerx = x + w / 2
        centery = y + h / 2
        nx = int(centerx - r)
        ny = int(centery - r)
        nr = int(r * 2)

        face = frame[ny:ny+nr, nx:nx+nr]
        face = cv2.resize(face, (160, 160))

        # Create embedding
        embedding = create_embedding(face)
        embedding1 = np.expand_dims(embedding, axis = 0)
        embedding_t = in_encoder.transform(embedding)
        
        # Predict
        check_mask = mask_model.predict(embedding_t)
        if check_mask == 1:
            model_classifier = wmask_model
        else:
            model_classifier = womask_model
            
        yhat_class = model_classifier.predict(embedding_t)
        yhat_prob = model_classifier.decision_function(embedding_t)[0]/sum(model_classifier.decision_function(embedding_t)[0])
        
        # Get name
        class_index = yhat_class[0]
        class_probability = yhat_prob[class_index] * 100
        predict_name = out_encoder.inverse_transform(yhat_class)
        
        # Draw box and name
        color = COLORS[class_index, :].tolist()
        cv2.rectangle(frame, (x, y), (x1, y1), color, 2)
        face_name = '%s (%.3f)' % (predict_name[0], class_probability)
        cv2.putText(frame, face_name, (x, y1 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
        
    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()